In [1]:
import time
import pandas as pd
import pyspark as ps
import pyarrow as pa
from pyspark.sql import SparkSession

timestr = time.strftime("%Y%m%d-%H%M%S")
# from pyspark.sql.functions import col
# from pyspark.sql.functions import udf
# from pyspark.sql.types import StringType

In [2]:
url = "jdbc:postgresql://localhost:5432/reddit_comments"

properties = {
    "user": "admin",
    "password": "theLegend30$$",
    "driver": "org.postgresql.Driver"
}

spark = SparkSession\
    .builder\
    .appName("Incel Comments Extraction (01/2020 - 12/2020)")\
    .config("spark.jars", "/usr/local/postgres/postgresql-42.6.0.jar")\
    .config("spark.driver.memory", "10g")\
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
    .getOrCreate()
    
df = spark.read \
    .format("jdbc") \
    .option("url", url)\
    .option("dbtable", "reddit_2020_aggr") \
    .option("user", properties["user"]) \
    .option("password", properties["password"])\
    .option("driver", properties["driver"])\
    .load()


23/06/15 12:08:00 WARN Utils: Your hostname, scarletbase resolves to a loopback address: 127.0.1.1; using 192.168.0.154 instead (on interface enp5s0)
23/06/15 12:08:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/06/15 12:08:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df.createOrReplaceTempView("redd2020incel")
query = """SELECT * FROM redd2020incel GROUP BY id, author, subreddit, subreddit_type, body, score, controversiality, created_on, row;"""

reddit_df = spark.sql(query)

In [4]:
incel_reddit_cache = reddit_df.cache()

In [5]:
# Load RDD to Pandas DataFrame
incelDF = incel_reddit_cache.toPandas()

In [6]:
# Descrive the numerical columns
incelDF.describe()

,score,controversiality,row
count,109437.000000,109437.0,109437.000000
mean,0.598043,1.0,1.002102
std,4.414665,0.0,0.045796
min,-64.000000,1.0,1.000000
25%,-1.000000,1.0,1.000000
50%,0.000000,1.0,1.000000
75%,2.000000,1.0,1.000000
max,203.000000,1.0,2.000000


In [7]:
# First 10
incelDF.head(10)

,id,author,subreddit,subreddit_type,body,score,controversiality,created_on,row
0,fg0knts,aberdisco,Tinder,public,There are no girls on the internet. It's all f...,2,1,2020-01-30,1
1,fczl735,aceinthehole001,relationships,public,&gt;it sounds like you are digging your heels ...,2,1,2020-01-03,1
2,ffbq8po,ALTSuzzxingcoh,rant,public,&gt; You think America is falling apart? My co...,-1,1,2020-01-23,1
3,fedtkif,big_chungo420,Advice,public,"Growing up, my best friend's mom was against v...",-5,1,2020-01-14,1
4,fdri6mn,brother1957,DeadBedrooms,public,The #metomovement has effected him and he does...,0,1,2020-01-10,1
5,fevamuz,charliekindafine,relationships,public,It seems like you guys aren't sexually compati...,3,1,2020-01-18,1
6,fcp2ie0,chief_x2,pussypassdenied,public,Christians were actively lighting women and ki...,1,1,2019-12-31,1
7,fda7faw,DoYerThang,relationships,public,"For ME, I would not want to change my sexual e...",-2,1,2020-01-05,1
8,ffb88at,GerinX,Tinder,public,I like to think you’re witticisms just fly ove...,-2,1,2020-01-22,1
9,fdmf36m,habez21,relationships,public,If you guys share finances then your fiancé ki...,1,1,2020-01-09,1


In [8]:
# Last 10
incelDF.tail(10)

,id,author,subreddit,subreddit_type,body,score,controversiality,created_on,row
109427,ggzy5a5,cherryteapie,Tinder,public,Is it necessary to insult/ be rude to someone ...,2,1,2020-12-25,1
109428,gh07rsy,MuffinPuff,self,public,None of what you've said is relevant to what I...,-4,1,2020-12-25,1
109429,gh0gw72,barn6758,dating,public,Lol i mean why did you think guys date younger...,-1,1,2020-12-25,1
109430,gh2ons1,fatkc,Advice,public,"Go with them, you'll be fine. These guidelines...",-1,1,2020-12-26,1
109431,ghb18je,UncutCoconut,dating_advice,public,Do you think it’s more of No.2. What about No....,-1,1,2020-12-28,1
109432,ghfi855,karamjar,Tinder,public,"Im genuinely interested, what are some todays ...",-14,1,2020-12-29,1
109433,ghfqv0c,SuperbClick,Tinder,public,A man with standard what?!?!\n\nAny man with s...,-2,1,2020-12-29,1
109434,ghgdrt7,soupz,relationships,public,I have to disappoint you and say that my forme...,3,1,2020-12-29,1
109435,ghi0j9p,hilfnafl,sex,public,You have the whole house to yourself so you ca...,2,1,2020-12-30,1
109436,ghmdmzw,GrandRub,datingoverthirty,public,"no one wants a woman whos biggest dream is ""to...",-3,1,2020-12-31,1


In [9]:
incelDF.to_csv('./data/RAW_incelDF-'+timestr+'.csv', columns=['id', 'author', 'subreddit', 'subreddit_type', 'body', 'score', 'controversiality', 'created_on'], index=False)